In [ ]:
%cd /workspaces/Pointcept
from pointcept.engines.defaults import  default_setup
from pointcept.engines.defaults import (
    default_argument_parser,
    default_config_parser,
)
from pointcept.engines.train import TRAINERS
import torch
import numpy as np

In [ ]:
args = default_argument_parser().parse_args(
    "--config-file /workspaces/Pointcept/configs/scannet/pipeline-sculpting-spunet.py --num-gpus 0 --options num_workers=0 num_worker_per_gpu=0 batch_size=1".split()
)
cfg = default_config_parser(args.config_file, args.options)

In [ ]:
cfg = default_setup(cfg)
#cfg.num_worker_per_gpu=0
trainer = TRAINERS.build(dict(type=cfg.train.type, cfg=cfg))

In [ ]:
train_loader = trainer.train_loader
i, b = next(enumerate(train_loader))

b

# Example scene & parameters

In [ ]:
noise_p0 = b['coord'][np.random.randint(0,len(b['coord']))]
noise_size=0.3
noise_cell_size=0.02
noise_num_cells=int(noise_size//noise_cell_size)

# Perlin

In [ ]:
!python -m pip install git+https://github.com/pvigier/perlin-numpy


In [ ]:
from perlin_numpy import (
    generate_fractal_noise_2d, generate_fractal_noise_3d,
    generate_perlin_noise_2d, generate_perlin_noise_3d
)
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
noise = generate_perlin_noise_3d(
    (noise_num_cells,noise_num_cells,noise_num_cells),
    (2, 2, 2), tileable=(False,False,False)
)

noise

i,j,k=np.indices(noise.shape)

noise=noise.flatten()
i=i.flatten()
j=j.flatten()
k=k.flatten()

print(i)
print(j)
print(k)
print(noise)

In [ ]:
px.scatter_3d(x=i,y=j,z=k,color=noise, opacity=0.5)

In [ ]:
import pandas as pd

def make_thr_df(v, num=20, inverse=False, sliced=False, slice_width=1):
    thresholds = np.linspace(v.min(),v.max(),num=num)
    if sliced:
        sel = [ np.bitwise_and(v>tminus1,v<t) for tminus1,t in zip(thresholds,np.roll(thresholds, -1*slice_width)) ]
    elif inverse:
        sel = [v<t for t in thresholds]
    else:
        sel = [v>t for t in thresholds]
    i_=np.hstack([i[s] for s in sel])
    j_=np.hstack([j[s] for s in sel])
    k_=np.hstack([k[s] for s in sel])
    v_=np.hstack([v[s] for s in sel])
    t_=np.hstack([v[s]*0+t for s, t in zip(sel,thresholds)])
    df = pd.DataFrame(dict(i=i_,j=j_,k=k_,value=v_, t=t_))
    return df

def slider_3dplot(v, num=20, inverse=False, sliced=False, slice_width=1):
    df = make_thr_df(v, num, inverse, sliced, slice_width)
    return px.scatter_3d(df, x="i",y="j",z="k",color="value",animation_frame="t", range_color =[v.min(),v.max()])

In [ ]:
for n in range(1):
    noise = generate_perlin_noise_3d(
        (noise_num_cells*10,noise_num_cells*10,noise_num_cells*10),
        (2, 2, 2), tileable=(False,False,False)
    )
    i,j,k=np.indices(noise.shape)

    noise=noise.flatten()
    i=i.flatten()
    j=j.flatten()
    k=k.flatten()

    df = make_thr_df(noise, sliced=True)
    df.to_csv(f"perlin_{n}.csv", index=False)

In [ ]:
# ranges...
np.min(noise),np.max(noise)

In [ ]:
slider_3dplot(v=noise)

In [ ]:
slider_3dplot(v=noise, inverse=True)

In [ ]:
slider_3dplot(v=noise, sliced=True)

In [ ]:
slider_3dplot(v=noise, sliced=True, num=50, slice_width=4)

In [ ]:
noise_rand_full = np.random.rand(*noise.shape)
slider_3dplot(v=noise_rand_full, inverse=True)

In [ ]:
!python -m pip install opensimplex

In [ ]:
k

In [ ]:
import opensimplex

simplex_max_coord = 3
coord_range = (np.linspace(0,1,num=noise_num_cells)*simplex_max_coord)
noise=(
    opensimplex.noise3array( coord_range,coord_range,coord_range).flatten()
            )
print(noise.min(),noise.max())
noise[:2]

In [ ]:
slider_3dplot(v=noise, sliced=True)

In [ ]:
import scipy

slider_3dplot(v=np.abs(noise))

# Surface

In [ ]:

noise = generate_perlin_noise_3d(
    (noise_num_cells,noise_num_cells,noise_num_cells),
    (2, 2, 2), tileable=(False,False,False)
)
i,j,k=np.indices(noise.shape)

noise=noise.flatten()

locs = np.bitwise_and(noise<0.1,noise>-0.1)

i=noise_cell_size*i.flatten()[locs]+noise_p0[0].numpy()
j=noise_cell_size*j.flatten()[locs]+noise_p0[1].numpy()
k=noise_cell_size*k.flatten()[locs]+noise_p0[2].numpy()

x=b['coord'][:,0]
y=b['coord'][:,1]
z=b['coord'][:,2]

pd.DataFrame(dict(x=i,y=j,z=k,noise=noise[locs])).to_csv("cube.csv", index=False)
pd.DataFrame(dict(x=x,y=y,z=z)).to_csv("scene.csv", index=False)